Melakukan import beberapa library yang digunakan untuk model testing.

In [1]:
import tensorflow as tf
import base64
import json
import requests

Membuat fungsi paylooad untuk melakukan testing terhadap ML pipeline model yang telah dibuat. Fungsi ini didefinisikan untuk melakukan encoding text postingan tweet menyesuaikan format yang diharapkan model dengan menggunakan TensorFlow 'Example' dan base64 encoding.

Selanjutnya, dibuat suatu list yang etrdiri atas beberapa tweet baru untuk melakukan tes terhadap algoritma model, apakah telah sesuai untuk memprediksi kategori sifat tweet baru tersebut berdasarkan sifatnya.

Model endpoint berfungsi untuk mengspesifikasi URL endpoint dari model ML pipeline yang telah diimplementasikan sebelumnya menggunakan Docker.

Bagian akhir dari kode ini akan membuat request prediksi terhadap setiap tweet pada list yang ada untuk kemudian mendapatkan keluaran berupa isi tweet dan hasil klasifikasinya.

In [2]:
def create_suicide_detection_payload(tweet_text):
    feature_dict = {
        "Tweet": tf.train.Feature(bytes_list=tf.train.BytesList(value=[tweet_text.encode("utf-8")])),
    }
    
    example_proto = tf.train.Example(
        features=tf.train.Features(feature=feature_dict)
    ).SerializeToString()
    
    payload = [
        {
            "examples": {
                "b64": base64.b64encode(example_proto).decode()
            }
        }
    ]
    
    return json.dumps({
        "signature_name": "serving_default",
        "instances": payload
    })

tweet_texts = [
    "I can't take this anymore. Life is too painful.",
    "Feeling a bit down today, but hoping tomorrow will be better.",
    "Sometimes I just want to disappear and never come back.",
    "Having a rough day, but I'll push through it.",
    "Why does everything have to be so hard? I'm so tired of fighting."
]

model_endpoint = "http://localhost:8080/v1/models/suicide-detection-model:predict"

for tweet_text in tweet_texts:
    json_payload = create_suicide_detection_payload(tweet_text)
    
    response = requests.post(model_endpoint, data=json_payload, headers={"Content-Type": "application/json"})
    
    try:
        predictions = response.json().get("predictions")
        if predictions:
            prediction_score = predictions[0][0]
            classification = "Potential Suicide post" if prediction_score > 0.5 else "Not Suicide post"
        else:
            classification = "Error: No predictions found in response."
    except (ValueError, IndexError, KeyError) as e:
        classification = f"Error processing response: {str(e)}"
    
    print(f"Tweet: {tweet_text}\nClassification: {classification}\n")

Tweet: I can't take this anymore. Life is too painful.
Classification: Potential Suicide post

Tweet: Feeling a bit down today, but hoping tomorrow will be better.
Classification: Not Suicide post

Tweet: Sometimes I just want to disappear and never come back.
Classification: Potential Suicide post

Tweet: Having a rough day, but I'll push through it.
Classification: Not Suicide post

Tweet: Why does everything have to be so hard? I'm so tired of fighting.
Classification: Potential Suicide post

